# [Module 5.1] getRec() 통한 추천 리스트 얻기

이 노트북은 Module2에서 생성한 솔류션을 바탕으로 아래와 같은 작업을 합니다.
* 캠페인 생성
* 캠페인을 통해 특정 유저에 대한 추천 영화 리스트 얻기


In [1]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [5]:
%store -r

생성할 오브젝트의 끝에 임의의 숫자를 부여하기 위해 suffix 정의

## 샘플 추천 결과 얻기

캠페인이 활성화되면 추천 결과를 받을 수 있습니다. 먼저 컬렉션에서 임의의 사용자를 선택해야 합니다. 그런 다음, ID 대신 추천을 위해 영화 정보를 표시하는 몇 가지 헬퍼 함수를 만듭니다.

In [4]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [5]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())


#### HRNN GetRecommendations 호출

아래 코드 셀을 실행하면 특정 사용자에 대한 추천 사항이 표시되고 추천 영화 목록이 반환됩니다.

In [6]:
df=pd.read_csv(interaction_filename)

# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("USER: {}".format(user_id))

get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = hrnn_campaign_arn,
    userId = str(user_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", user_id)

item_list = get_recommendations_response['itemList']
recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
recommendations_df

USER: 2489
Recommendations for user:  2489


,OriginalRecs
0,"Shawshank Redemption, The (1994)"
1,North by Northwest (1959)
2,To Kill a Mockingbird (1962)
3,"Conversation, The (1974)"
4,Gone with the Wind (1939)
5,Amadeus (1984)
6,Chinatown (1974)
7,"Producers, The (1968)"
8,"Wizard of Oz, The (1939)"
9,"Hustler, The (1961)"


#### Sims GetRecommendations 호출
아래 코드 셀을 실행하면 특정 아이템과 유사한 추천 영화 목록이 반환됩니다.

In [7]:
# Getting a random user:
user_id, item_id, _,_,_ = df.sample().values[0]
print("ITEM ID: {}".format(item_id))


get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(item_id),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for item_id: ", item_id)

item_list = get_recommendations_response['itemList']
recommendation_title_list = []
recommendation_id_list=[]
for item in item_list:
    title = get_movie_title(item['itemId'])
    recommendation_title_list.append(title)
    recommendation_id_list.append(item['itemId'])
recommendations_df = pd.DataFrame(recommendation_title_list, columns = ['OriginalRecs'])
recommendations_df

ITEM ID: 589
Recommendations for item_id:  589


,OriginalRecs
0,Total Recall (1990)
1,Men in Black (1997)
2,Starship Troopers (1997)
3,Independence Day (ID4) (1996)
4,Die Hard 2 (1990)
5,Star Trek VI: The Undiscovered Country (1991)
6,"Lost World: Jurassic Park, The (1997)"
7,Heat (1995)
8,Patriot Games (1992)
9,Braveheart (1995)


## Amazon Personalize Batch Export 작업 


Amazon Personalize Batch기능을 활용하려면 json 파일 형식으로 추천 받고하 자는 사용자 또는 아이템 아이디를 json 형태의 파일로 s3에 저장하여야 합니다. Output의 형식도 json형태로 저장되며 지정한 S3 bucket 경로에 저장 되게 됩니다. 

HRNN 솔루션  Batch Input 예제: 

```JSON,
    {"userId": "4638"},
    {"userId": "663"},
    {"userId": "3384"},
```


Batch Output 예제: 
```JSON,
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```


In [6]:
#Get the user list
df=pd.read_csv(interaction_filename)
df_users = df['USER_ID'].unique()
df_users=pd.DataFrame(df_users,columns=['USER_ID'])


In [7]:
#Get the user list
#batch_users = df_users.sample(3).index.tolist()
batch_users=df_users.index.tolist()
data_dir="dataset/"
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir+json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [9]:
# Showcase the input file:
!head -n10 $data_dir$json_input_filename

{"userId": "0"}
{"userId": "1"}
{"userId": "2"}
{"userId": "3"}
{"userId": "4"}
{"userId": "5"}
{"userId": "6"}
{"userId": "7"}
{"userId": "8"}
{"userId": "9"}


In [10]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket).Object(data_dir+json_input_filename).upload_file(data_dir+json_input_filename)
s3_input_path = "s3://" + bucket + "/" + data_dir+json_input_filename
print(s3_input_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/json_input.json


In [11]:
# Define the output path
s3_output_path = "s3://" + bucket + "/"+data_dir
print(s3_output_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/


In [12]:
print(role_arn)

arn:aws:iam::057716757052:role/PersonalizeRoleDemo91891


In [13]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-HRNN-"+suffix,
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

In [ ]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  12:26:26 PM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS


In [ ]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket,data_dir+export_name,data_dir+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open("dataset/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

## 리뷰

캠페인을 생성하고 실제적으로 특정 유저의 추천 영화 목록도 얻었습니다.
이제 다음 노트북으로 넘어갈 준비가 되었습니다. (`4.View_Campaign_And_Interactions.ipynb`)


## 다음 노트북에 대한 참고 사항

다음 실습에 필요한 몇 가지 값들이 있습니다. 아래 셀을 실행하여 저장한 후, 다음 주피터 노트북에서 그대로 사용할 수 있습니다.

In [ ]:
%store hrnn_campaign_arn
%store hrnn_coldstart_campaign_arn
%store sims_campaign_arn
%store ranking_campaign_arn
%store recommendations_df
%store user_id